In [3]:
# !pip install --upgrade "transformers<5.0"

In [5]:
# !pip install git+https://github.com/intel/auto-round.git

In [2]:
import os
import torch
from auto_round import AutoRound
from huggingface_hub import HfApi, create_repo, notebook_login, get_token
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


PyTorch Version: 2.8.0+cu126
CUDA Available: True
CUDA Version: 12.6
GPU Name: NVIDIA RTX A5000
VRAM: 23.5 GB


In [6]:
MODEL_ID = "Qwen/Qwen3-VL-2B-Instruct"
HF_USER = "Vishva007"
OUTPUT_BASE_DIR = "./AutoRound"

In [3]:
notebook_login()

In [6]:
TUNING_CONFIG = {
    "group_size": 128,
    "sym": True,
    "iters": 800,  # High accuracy (Production grade)
    "nsamples": 512,  # More calibration data
    "batch_size": 2,  # Faster on 48GB VRAM
    "seqlen": 2048,
    "low_gpu_mem_usage": False,  # Keep on GPU for speed
    "enable_torch_compile": True,  # JIT acceleration
    "quant_nontext_module": False,  # Keep Vision Tower in BF16 (Crucial for VLM accuracy)
}

In [10]:
def push_to_hub(local_dir, repo_name, token):
    """Creates repo and uploads folder to Hugging Face."""
    full_repo_id = f"{HF_USER}/{repo_name}"
    print(f"\n[Hub] Pushing {local_dir} to {full_repo_id}...")

    try:
        api = HfApi()
        create_repo(
            full_repo_id, repo_type="model", exist_ok=True, private=False, token=token
        )

        api.upload_folder(
            folder_path=local_dir, repo_id=full_repo_id, repo_type="model", token=token
        )
        print(f"[Hub] ✅ Successfully uploaded: https://huggingface.co/{full_repo_id}")
    except Exception as e:
        print(f"[Hub] ❌ Error uploading: {e}")

In [ ]:
model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_ID, dtype=torch.float16, device_map="auto"
)
processor = AutoProcessor.from_pretrained(MODEL_ID)

tokenizer = processor.tokenizer


In [9]:
ar = AutoRound(
    model=model,
    tokenizer=tokenizer,
    processor=processor,
    scheme="W4A16",
    model_dtype="fp16",
    **TUNING_CONFIG,
)

2026-02-07 16:28:01 INFO autoround.py L165: using MLLM mode for multimodal model.
2026-02-07 16:28:02 INFO base.py L486: using torch.float16 for quantization tuning


In [10]:
# SINGLE CALL to save all 3 formats to the same output directory
# The files will exist side-by-side or merged in this folder.
ar.quantize_and_save(
    OUTPUT_BASE_DIR, format="auto_gptq,auto_awq,auto_round", inplace=True
)

2026-02-07 16:28:18 WARNING base.py L1639: `low_cpu_mem_usage` is only supported when `immediate_packing` is True. Setting `low_cpu_mem_usage` to False.
2026-02-07 16:28:18 INFO base.py L1739: start to cache block inputs


README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

data/train-00000-of-00001-4746b8785c874c(…):   0%|          | 0.00/33.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1216 [00:00<?, ? examples/s]

cache block inputs:   0%|          | 0/512 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/models/qwen3_vl/modeling_qwen3_vl.py:328: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:233.)
  freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(2, 3)
cache block inputs: 100%|██████████| 512/512 [00:11<00:00, 45.04it/s]
2026-02-07 16:29:29 INFO base.py L1754: caching done
Quantizing model.language_model.laye

(Qwen3VLForConditionalGeneration(
   (model): Qwen3VLModel(
     (visual): Qwen3VLVisionModel(
       (patch_embed): Qwen3VLVisionPatchEmbed(
         (proj): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
       )
       (pos_embed): Embedding(2304, 1024)
       (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
       (blocks): ModuleList(
         (0-23): 24 x Qwen3VLVisionBlock(
           (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
           (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
           (attn): Qwen3VLVisionAttention(
             (qkv): Linear(in_features=1024, out_features=3072, bias=True)
             (proj): Linear(in_features=1024, out_features=1024, bias=True)
           )
           (mlp): Qwen3VLVisionMLP(
             (linear_fc1): Linear(in_features=1024, out_features=4096, bias=True)
             (linear_fc2): Linear(in_features=4096, out_features=1024, bias=True)
             (act_fn): GELUTanh()
           

In [7]:
base_name = MODEL_ID.split("/")[-1]
hf_token = get_token()

In [8]:
path_autoround = os.path.join(OUTPUT_BASE_DIR, "auto-round-auto-gptq")
path_gptq = os.path.join(OUTPUT_BASE_DIR, "auto-gptq")
path_awq = os.path.join(OUTPUT_BASE_DIR, "auto-awq")

In [11]:
if hf_token:
    # 1. AutoRound Repo
    # Verify path exists before uploading
    if os.path.exists(path_autoround):
        push_to_hub(path_autoround, f"{base_name}-W4A16-AutoRound", hf_token)
    else:
        print(f"⚠️ Could not find AutoRound output at {path_autoround}")

    # 2. GPTQ Repo
    if os.path.exists(path_gptq):
        push_to_hub(path_gptq, f"{base_name}-W4A16-AutoRound-GPTQ", hf_token)
    else:
        print(f"⚠️ Could not find GPTQ output at {path_gptq}")

    # 3. AWQ Repo
    if os.path.exists(path_awq):
        push_to_hub(path_awq, f"{base_name}-W4A16-AutoRound-AWQ", hf_token)
    else:
        print(f"⚠️ Could not find AWQ output at {path_awq}")


[Hub] Pushing ./AutoRound/auto-round-auto-gptq to Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[Hub] ✅ Successfully uploaded: https://huggingface.co/Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound

[Hub] Pushing ./AutoRound/auto-gptq to Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound-GPTQ...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[Hub] ✅ Successfully uploaded: https://huggingface.co/Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound-GPTQ

[Hub] Pushing ./AutoRound/auto-awq to Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound-AWQ...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[Hub] ✅ Successfully uploaded: https://huggingface.co/Vishva007/Qwen3-VL-2B-Instruct-W4A16-AutoRound-AWQ
